In [ ]:
# I/O
import os
os.environ["OMP_NUM_THREADS"] = "1" # export OMP_NUM_THREADS=1
os.environ["OPENBLAS_NUM_THREADS"] = "1" # export OPENBLAS_NUM_THREADS=1
os.environ["MKL_NUM_THREADS"] = "1" # export MKL_NUM_THREADS=1
os.environ["VECLIB_MAXIMUM_THREADS"] = "1" # export VECLIB_MAXIMUM_THREADS=1
os.environ["NUMEXPR_NUM_THREADS"] = "1" # export NUMEXPR_NUM_THREADS=1

# Dataset
import numpy as np
import netCDF4 as nc

# Plot
import matplotlib.cm as cm
import matplotlib.pyplot as plt

# Time
import time

In [ ]:
def filename_constructor(time_index, case, var):
    return str(f"/data/cloud2023/vvmData{case}/archive{var}-{str(time_index).zfill(6)}.nc")

casenames = ["/rce_walker_15k_05m_p3",
             "/rce_walker_15k_1m_p3",
             "/rce_walker_1k_1m_p3",
             "/rce_walker_1k_2m_p3"]
varnames = ["/exp.C.LandSurface",
            "/exp.C.Surface",
            "/exp.L.Dynamic",
            "/exp.L.Radiation",
            "/exp.L.Thermodynamic"]
os.getcwd()

In [ ]:
def vertical_integrator(f, x):
    return 0

In [ ]:
class output:
    def __init__(self, case_idx):
        self.ID = case_idx
        self.PW_MEAN = []
        self.CRH_MEAN = []
        self.LWP_MEAN = []
        self.IWP_MEAN = []
        self.SST_MEAN = []
        self.Precip_MEAN = []
        self.PW_VARIANCE = []
        self.CRH_VARIANCE = []
        self.LWP_VARIANCE = []
        self.IWP_VARIANCE = []
        self.SST_VARIANCE = []
        self.Precip_VARIANCE = []
        return None
    def reader(self,time):
        # Caluclate saturated water vapour pressure
        def calculate_pw(temperature):
            temp = np.zeros_like(temperature)
            temp[temperature>96] = 6.112*np.exp((17.67*(temperature[temperature>96]-273.15))/(temperature[temperature>96]+243.5-273.15))
            return temp
        def calculate_qvs(pressure, temperature):
            temp = calculate_pw(temperature)
            return (0.622*temp)/(pressure - temp)
        
        # Read variables
        dataset = nc.Dataset(filename_constructor(str(time).zfill(6), casenames[self.ID], varnames[-1]))
        xc = dataset.variables["xc"][:]
        yc = dataset.variables["yc"][:]
        zc = dataset.variables["zc"][:]
        th = dataset.variables["th"][-1, :, :, :]
        qv = dataset.variables["qv"][-1, :, :, :]
        qc = dataset.variables["qc"][-1, :, :, :]
        qi = dataset.variables["qi"][-1, :, :, :]
        dataset = nc.Dataset(filename_constructor(str(time).zfill(6), casenames[self.ID], varnames[1]))
        precip = dataset.variables["sprec"][-1, :, :] * 3600
        sst = dataset.variables["tg"][-1, :, :]
        
        # Read fort.98
        RHO = np.loadtxt(f"/data/cloud2023/vvmData{casenames[-1]}/fort.98", 
                         dtype=float, 
                         skiprows=237,
                         usecols=1,
                         unpack=True,
                         max_rows=46)
        RHO = (RHO[1:]+RHO[:-1])/2
        RHO = RHO.reshape(-1,1,1)
        PBAR = np.loadtxt(f"/data/cloud2023/vvmData{casenames[-1]}/fort.98", 
                      dtype=float, 
                      skiprows=237,
                      usecols=3,
                      unpack=True,
                      max_rows=46)
        PBAR = (PBAR[1:]+PBAR[:-1])/2
        PBAR = PBAR.reshape(-1,1,1)/100
        PIBAR = np.loadtxt(f"/data/cloud2023/vvmData{casenames[-1]}/fort.98", 
                          dtype=float, 
                          skiprows=237,
                          usecols=4,
                          unpack=True,
                          max_rows=46)
        PIBAR = (PIBAR[1:]+PIBAR[:-1])/2
        PIBAR = PIBAR.reshape(-1,1,1)
        
        pw = np.trapz(RHO*qv, zc, axis = 0)
        temp = np.trapz(RHO*calculate_qvs(PBAR, th*PIBAR), zc, axis = 0)
        crh = pw/temp*100
        crh[crh>100] = 100
        crh[crh<0] = 0
        lwp = np.trapz(RHO*qc, zc, axis = 0)
        iwp = np.trapz(RHO*qi, zc, axis = 0)
        
        # append domain averages
        self.PW_MEAN.append(np.mean(pw))
        self.CRH_MEAN.append(np.mean(crh))
        self.LWP_MEAN.append(np.mean(lwp))
        self.IWP_MEAN.append(np.mean(iwp))
        self.SST_MEAN.append(np.mean(sst))
        self.Precip_MEAN.append(np.mean(precip))
        # append domain variances
        self.PW_VARIANCE.append(np.var(pw))
        self.CRH_VARIANCE.append(np.var(crh))
        self.LWP_VARIANCE.append(np.var(lwp))
        self.IWP_VARIANCE.append(np.var(iwp))
        self.SST_VARIANCE.append(np.var(sst))
        self.Precip_VARIANCE.append(np.var(precip))
        return crh
    def export(self):
        MEAN = np.array([self.PW_MEAN, 
                         self.CRH_MEAN, 
                         self.LWP_MEAN, 
                         self.IWP_MEAN,
                         self.SST_MEAN,
                         self.Precip_MEAN])
        VARIANCE = np.array([self.PW_VARIANCE, 
                             self.CRH_VARIANCE, 
                             self.LWP_VARIANCE, 
                             self.IWP_VARIANCE,
                             self.SST_VARIANCE,
                             self.Precip_VARIANCE])
        return MEAN, VARIANCE

In [ ]:
rce_walker_15k_05m_p3 = output(case_idx = 0)
rce_walker_15k_1m_p3 = output(case_idx = 1)

time = np.arange(0,1200)

for i in time:
    print(i)
    rce_walker_15k_05m_p3.reader(i)
    rce_walker_15k_1m_p3.reader(i)
naggregate_mean, naggregate_var = rce_walker_15k_05m_p3.export()
aggregate_mean, aggregate_var = rce_walker_15k_1m_p3.export()

In [ ]:
fig, axes = plt.subplots(nrows = 3,
                         ncols = 2,
                         sharex = True,
                         figsize=(16, 9),
                         dpi = 160)
fig.suptitle("Mean", fontsize = 22)
for idx, ax in enumerate(axes.flatten()):
    ax.plot(time, aggregate_mean[idx,:], 
            color = "r",
            linestyle = "solid",
            alpha = 0.5, 
            label = "15k_1m")
    ax.plot(time, naggregate_mean[idx,:], 
            color = "b",
            linestyle = "solid",
            alpha = 0.5, 
            label = "15k_05m")
    match idx:
        case 0:
            ax.legend(loc = 2)
            ax.set_ylabel("$(kg/m^2)$", fontsize = 12)
            ax.set_ylim(0, 100)
            ax.set_title("Precipitable Water", fontsize = 18)
            pass
        case 1:
            ax.legend(loc = 2)
            ax.set_ylabel("(%)", fontsize = 12)
            ax.set_ylim(0, 100)
            ax.set_title("Column Relative Humidity", fontsize = 18)
            pass
        case 2:
            ax.legend(loc = 2)
            ax.set_ylabel("$(kg/m^2)$", fontsize = 12)
            ax.set_ylim(0, 0.1)
            ax.set_title("Liquid Water Path", fontsize = 18)
            pass
        case 3:
            ax.legend(loc = 2)
            ax.set_ylabel("$(kg/m^2)$", fontsize = 12)
            ax.set_ylim(0, 1)
            ax.set_title("Ice Water Path", fontsize = 18)
            pass
        case 4:
            ax.legend(loc = 2)
            ax.set_xlabel("Day")
            ax.set_ylabel("$(K)$", fontsize = 12)
            ax.set_ylim(297.5, 307.5)
            ax.set_title("SST", fontsize = 18)
            pass
        case 5:
            ax.legend(loc = 2)
            ax.set_xlabel("Day")
            ax.set_xticks(np.linspace(0,1200,6), ["0", "10", "20", "30", "40", "50"])
            ax.set_xlim(0,1200)
            ax.set_ylabel("$(mm/hr)$", fontsize = 12)
            ax.set_ylim(0, 1)
            ax.set_title("Precipitation", fontsize = 18)
            pass
plt.savefig("MEAN.png")

In [ ]:
fig, axes = plt.subplots(nrows = 3,
                         ncols = 2,
                         sharex = True,
                         figsize=(16, 9),
                         dpi = 160)
fig.suptitle("Variance", fontsize = 22)
for idx, ax in enumerate(axes.flatten()):
    ax.plot(time, aggregate_var[idx,:], 
            color = "r",
            linestyle = "solid",
            alpha = 0.5, 
            label = "15k_1m")
    ax.plot(time, naggregate_var[idx,:], 
            color = "b",
            linestyle = "solid",
            alpha = 0.5, 
            label = "15k_05m")
    match idx:
        case 0:
            ax.legend(loc = 2)
            ax.set_ylabel("$(kg/m^2)^2$", fontsize = 12)
            ax.set_ylim(0, 600)
            ax.set_title("Precipitable Water", fontsize = 18)
            pass
        case 1:
            ax.legend(loc = 2)
            ax.set_ylabel("(%)^2", fontsize = 12)
            ax.set_ylim(0, 1000)
            ax.set_title("Column Relative Humidity", fontsize = 18)
            pass
        case 2:
            ax.legend(loc = 2)
            ax.set_ylabel("$(kg/m^2)^2$", fontsize = 12)
            ax.set_ylim(0, 0.1)
            ax.set_title("Liquid Water Path", fontsize = 18)
            pass
        case 3:
            ax.legend(loc = 2)
            ax.set_ylabel("$(kg/m^2)^2$", fontsize = 12)
            ax.set_ylim(0, 5)
            ax.set_title("Ice Water Path", fontsize = 18)
            pass
        case 4:
            ax.legend(loc = 2)
            ax.set_xlabel("Day")
            ax.set_ylabel("$(K)^2$", fontsize = 12)
            ax.set_ylim(0, 1)
            ax.set_title("SST", fontsize = 18)
            pass
        case 5:
            ax.legend(loc = 2)
            ax.set_xlabel("Day")
            ax.set_xticks(np.linspace(0,1200,6), ["0", "10", "20", "30", "40", "50"])
            ax.set_xlim(0,1200)
            ax.set_ylabel("$(mm/hr)^2$", fontsize = 12)
            ax.set_ylim(0, 15)
            ax.set_title("Precipitation", fontsize = 18)
            pass
plt.savefig("VARIANCE.png")